In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
import pickle
from sklearn.ensemble import RandomForestClassifier

In [2]:
categorical = pd.read_csv('categorical.csv')
numerical = pd.read_csv('numerical.csv')
target = pd.read_csv('target.csv')

In [3]:
data = pd.concat([numerical, categorical, target], axis = 1)
display(data.shape)
data.head()

(95412, 339)

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
0,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11.0,0,0.0
1,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10.0,0,0.0
2,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5

### TARGET_D IS A REGRESSION COLUMN => Too many different values
### TARGET_B IS THE CLASIIFICATION TARGET => Only Two Unique Values

In [4]:
#target['TARGET_D'].value_counts()

In [5]:
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [6]:
cols = []
for col in data.columns:
    cols.append(col.lower())
data.columns = cols
print(data.columns)

Index(['tcode', 'age', 'income', 'wealth1', 'hit', 'malemili', 'malevet',
       'vietvets', 'wwiivets', 'localgov',
       ...
       'minrdate_yr', 'minrdate_mm', 'maxrdate_yr', 'maxrdate_mm',
       'lastdate_yr', 'lastdate_mm', 'firstdate_yr', 'firstdate_mm',
       'target_b', 'target_d'],
      dtype='object', length=339)


In [7]:
data = data.drop('target_d', axis =1)
y = data['target_b']
X = data.drop(['target_b'], axis = 1)
numerical = X.select_dtypes(np.number).reset_index(drop=True)
categorical = X.select_dtypes(object).reset_index(drop=True)

In [8]:
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first').fit(categorical)
encoded_categorical = encoder.transform(categorical).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)

final_data = pd.concat([numerical, encoded_categorical], axis = 1)
final_data.head()

,tcode,age,income,wealth1,hit,malemili,malevet,vietvets,wwiivets,localgov,stategov,fedgov,wealth2,pop901,pop902,pop903,pop90c1,pop90c2,pop90c3,pop90c4,pop90c5,eth1,eth2,eth3,eth4,eth5,eth6,eth7,eth8,eth9,eth10,eth11,eth12,eth13,eth14,eth15,eth16,age901,age902,age903,age904,age905,age906,age907,chil1,chil2,chil3,agec1,agec2,agec3,agec4,agec5,agec6,agec7,chilc1,chilc2,chilc3,chilc4,chilc5,hhage1,hhage2,hhage3,hhn1,hhn2,hhn3,hhn4,hhn5,hhn6,marr1,marr2,marr3,marr4,hhp1,hhp2,dw1,dw2,dw3,dw4,dw5,dw6,dw7,dw8,dw9,hv1,hv2,hv3,hv4,hu1,hu2,hu3,hu4,hu5,hhd1,hhd2,hhd3,hhd4,hhd5,hhd6,hhd7,hhd8,hhd9,hhd10,hhd11,hhd12,ethc1,ethc2,ethc3,ethc4,ethc5,ethc6,hvp1,hvp2,hvp3,hvp4,hvp5,hvp6,hur1,hur2,rhp1,rhp2,rhp3,rhp4,hupa1,hupa2,hupa3,hupa4,hupa5,hupa6,hupa7,rp1,rp2,rp3,rp4,msa,adi,dma,ic1,ic2,ic3,ic4,ic5,ic6,ic7,ic8,ic9,ic10,ic11,ic12,ic13,ic14,ic15,ic16,ic17,ic18,ic19,ic20,ic21,ic22,ic23,hhas1,hhas2,hhas3,hhas4,mc1,mc2,mc3,tpe1,tpe2,tpe3,tpe4,tpe5,tpe6,tpe7,tpe8,tpe9,pec1,pec2,tpe10,tpe11,tpe12,tpe13,lfc1,lfc2,lfc3,lfc4,lfc5,lfc6,lfc7,lfc8,lfc9,lfc10,occ1,occ2,occ3,occ4,occ5,occ6,occ7,occ8,occ9,occ10,occ11,occ12,occ13,eic1,eic2,eic3,eic4,eic5,eic6,eic7,eic8,eic9,eic10,eic11,eic12,eic13,eic14,eic15,eic16,oedc1,oedc2,oedc3,oedc4,oedc5,oedc6,oedc7,ec1,ec2,ec3,ec4,ec5,ec6,ec7,ec8,sec1,sec2,sec3,sec4,sec5,afc1,afc2,afc3,afc4,afc5,afc6,vc1,vc2,vc3,vc4,anc1,anc2,anc3,anc4,anc5,anc6,anc7,anc8,anc9,anc10,anc11,anc12,anc13,anc14,anc15,pobc1,pobc2,lsc1,lsc2,lsc3,lsc4,voc1,voc2,voc3,hc1,hc2,hc3,hc4,hc5,hc6,hc7,hc8,hc9,hc10,hc11,hc12,hc13,hc14,hc15,hc16,hc17,hc18,hc19,hc20,hc21,mhuc1,mhuc2,ac1,ac2,cardprom,numprom,cardpm12,numprm12,ramntall,ngiftall,cardgift,minramnt,maxramnt,lastgift,timelag,avggift,controln,hphone_d,rfa_2f,cluster2,cluster,datasrce,domain_b,odatew_yr,odatew_mm,dob_yr,dob_mm,minrdate_yr,minrdate_mm,maxrdate_yr,maxrdate_mm,lastdate_yr,lastdate_mm,firstdate_yr,firstdate_mm,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39,36,3,2,89,1,37,12,92,8,94,2,95,12,89,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1,14,3,1,94,1,52,2,93,10,95,12,95,12,93,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0

In [9]:
X_train, X_test, y_train, y_test = train_test_split(final_data, y, test_size=0.25, random_state=1900)

In [10]:
print(X_train.shape)
print(X_test.shape)
my_df = pd.concat((X_train,y_train), axis = 1)
print(my_df.shape)

(71559, 354)
(23853, 354)
(71559, 355)


In [11]:
my_df.head()

,tcode,age,income,wealth1,hit,malemili,malevet,vietvets,wwiivets,localgov,stategov,fedgov,wealth2,pop901,pop902,pop903,pop90c1,pop90c2,pop90c3,pop90c4,pop90c5,eth1,eth2,eth3,eth4,eth5,eth6,eth7,eth8,eth9,eth10,eth11,eth12,eth13,eth14,eth15,eth16,age901,age902,age903,age904,age905,age906,age907,chil1,chil2,chil3,agec1,agec2,agec3,agec4,agec5,agec6,agec7,chilc1,chilc2,chilc3,chilc4,chilc5,hhage1,hhage2,hhage3,hhn1,hhn2,hhn3,hhn4,hhn5,hhn6,marr1,marr2,marr3,marr4,hhp1,hhp2,dw1,dw2,dw3,dw4,dw5,dw6,dw7,dw8,dw9,hv1,hv2,hv3,hv4,hu1,hu2,hu3,hu4,hu5,hhd1,hhd2,hhd3,hhd4,hhd5,hhd6,hhd7,hhd8,hhd9,hhd10,hhd11,hhd12,ethc1,ethc2,ethc3,ethc4,ethc5,ethc6,hvp1,hvp2,hvp3,hvp4,hvp5,hvp6,hur1,hur2,rhp1,rhp2,rhp3,rhp4,hupa1,hupa2,hupa3,hupa4,hupa5,hupa6,hupa7,rp1,rp2,rp3,rp4,msa,adi,dma,ic1,ic2,ic3,ic4,ic5,ic6,ic7,ic8,ic9,ic10,ic11,ic12,ic13,ic14,ic15,ic16,ic17,ic18,ic19,ic20,ic21,ic22,ic23,hhas1,hhas2,hhas3,hhas4,mc1,mc2,mc3,tpe1,tpe2,tpe3,tpe4,tpe5,tpe6,tpe7,tpe8,tpe9,pec1,pec2,tpe10,tpe11,tpe12,tpe13,lfc1,lfc2,lfc3,lfc4,lfc5,lfc6,lfc7,lfc8,lfc9,lfc10,occ1,occ2,occ3,occ4,occ5,occ6,occ7,occ8,occ9,occ10,occ11,occ12,occ13,eic1,eic2,eic3,eic4,eic5,eic6,eic7,eic8,eic9,eic10,eic11,eic12,eic13,eic14,eic15,eic16,oedc1,oedc2,oedc3,oedc4,oedc5,oedc6,oedc7,ec1,ec2,ec3,ec4,ec5,ec6,ec7,ec8,sec1,sec2,sec3,sec4,sec5,afc1,afc2,afc3,afc4,afc5,afc6,vc1,vc2,vc3,vc4,anc1,anc2,anc3,anc4,anc5,anc6,anc7,anc8,anc9,anc10,anc11,anc12,anc13,anc14,anc15,pobc1,pobc2,lsc1,lsc2,lsc3,lsc4,voc1,voc2,voc3,hc1,hc2,hc3,hc4,hc5,hc6,hc7,hc8,hc9,hc10,hc11,hc12,hc13,hc14,hc15,hc16,hc17,hc18,hc19,hc20,hc21,mhuc1,mhuc2,ac1,ac2,cardprom,numprom,cardpm12,numprm12,ramntall,ngiftall,cardgift,minramnt,maxramnt,lastgift,timelag,avggift,controln,hphone_d,rfa_2f,cluster2,cluster,datasrce,domain_b,odatew_yr,odatew_mm,dob_yr,dob_mm,minrdate_yr,minrdate_mm,maxrdate_yr,maxrdate_mm,lastdate_yr,lastdate_mm,firstdate_yr,firstdate_mm,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,target_b
1662,4,75.000000,7,9,12,0,39,6,57,5,9,0,9,861,255,338,99,0,0,46,54,81,2,0,16,4,0,1,10,3,1,0,0,1,0,0,3,46,52,53,45,53,55,17,44,39,17,7,9,21,17,15,16,14,19,19,33,11,18,44,11,41,16,45,39,20,9,2,66,6,11,17,175,254,99,99,0,0,0,0,0,0,0,5615,5373,12,9,95,5,97,3,0,25,75,67,22,85,15,2,1,1,9,22,8,11,41,29,0,1,0,98,98,98,98,99,95,0,91,70,72,14,3,0,0,0,5,0,0,0,75,75,75,75,7360.0,65.0,807.0,785,996,849,1017,44204,3,7,10,16,11,16,3,14,20,0,3,7,11,11,18,4,19,27,39,0,64,1,29,71,6,60,8,18,3,15,0,0,6,8,0,16,22,27,6,81,56,73,40,73,40,40,40,99,0,0,41,12,3,23,15,4,2,0,0,0,0,0,0,0,0,0,1,5,0,1,9,31,4,4,0,23,10,8,2,6,2,0,27,52,13,0,160,2,8,14,17,8,25,26,19,3,2,15,5,0,0,0,20,36,6,21,42,50,9,0,11,6,3,2,1,14,3,1,2,0,3,0,0,1,21,54,73,4,8,14,93,67,22,18,51,0,0,0,5,12,88,0,0,90,0,6,0,0,4,99,0,99,99,99,19,3,9,9,26,62,6,14,118.0,9,8,5.0,25.0,11.0,9,13.111111,163558,1,2,1,13,3,1,86,1,23,1,92,9,87,5,96,2,86,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
75828,1,46.000000,4,5,2,0,45,43,18,7,5,5,5,1420,385,443,99,0,0,49,51,81,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,38,41,31,41,44,32,30,46,24,14,24,29,15,9,7,2,11,11,38,23,17,14,3,12,10,26,64,42,16,5,61,10,4,26,263,320,99,99,0,0,0,0,0,0,0,493,553,4,3,84,16,96,4,6,55,87,70,42,93,7,13,2,11,8,10,3,20,54,8,6,11,1,0,1,4,12,48,0,0,68,60,64,18,5,0,0,0,16,0,0,0,1,32,85,88,4520.0,209.0,529.0,345,346,350,362,10800,19,14,19,24,24,0,0,0,0,15,14,22,25,23,0,0,0,0,19,3,41,11,41,59,15,82,15,0,0,0,0,0,0,3,48,6,20,21,0,76,70,77,64,75,63,77,57,67,99,11,12,9,8,10,23,0,0,8,1,13,6,3,6,0,1,12,14,6,5,4,15,6,4,4,1,10,8,6,2,7,5,5,3,75,5,0,120,8,15,32,24,9,6,5,3,30,3,24,6,0,0,0,22,45,0,43,3,18,16,2,5,0,12,0,0,10,1,0,0,0,0,1,0,0,0,40,97,2,0,1,97,74,20,10,19,0,3,12,58,80,20,0,0,78,0,16,7,0,0,98,2,96,99,98,5,2,6,2,23,58,6,13,106.5,10,6,5.0,16.0,16.0,4,10.650000,62917,1,1,22,18,3,2,90,1,52,1,94,4,94,2,95,11,91,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
47947,2,61.611649,5,9,0,0,25,6,54,1,5,7,2,4162,1113,1383,0,0,99,

## UPSCALING DATA

In [12]:
from sklearn.utils import resample

In [13]:
cat_0 = my_df[my_df['target_b'] == 0]
cat_1 = my_df[my_df['target_b'] == 1]

In [14]:
print(cat_0.shape)
print(cat_1.shape)

(67930, 355)
(3629, 355)


In [15]:
cat_1_oversampled = resample(cat_1,  replace=True,  n_samples = len(cat_0))
cat_1_oversampled.shape                                
                                 

(67930, 355)

In [16]:
upsampled_data = pd.concat([cat_1_oversampled,cat_0], axis = 0)
upsampled_data.shape

(135860, 355)

## Fill NaN value in 'firstdate_mm' with the most common value

In [17]:
upsampled_data['firstdate_mm'].value_counts(dropna = False)

1.0     25454
9.0     21658
10.0    20426
2.0     18049
3.0      9686
11.0     9170
12.0     8415
4.0      6302
6.0      4839
8.0      4796
7.0      4104
5.0      2960
NaN         1
Name: firstdate_mm, dtype: int64

In [18]:
upsampled_data['firstdate_mm'] = upsampled_data['firstdate_mm'].fillna(1.0)

## Applying Random Forest to the Upscaled data

In [19]:
y_upsampled = upsampled_data['target_b']
X_upsampled = upsampled_data.drop(['target_b'], axis = 1)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_upsampled, y_upsampled, test_size=0.3, random_state=1500)

In [21]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=10,
                             min_samples_split=20,
                             min_samples_leaf =20)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

0.8515909234295809
0.828328180970607


In [22]:
from sklearn.metrics import accuracy_score
predicted = clf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)
accuracy

0.828328180970607

In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,predicted)

array([[16281,  4019],
       [ 2978, 17480]], dtype=int64)

# THIS MODEL IS TERRIBLE. SHALL IMPROVE IT LATER

# LAB 2. PREDICTING THE AMOUNT
## Using data before Upscaling

In [24]:
donors = my_df[my_df['target_b'] == 1]
donors.shape

(3629, 355)

In [25]:
for col in donors.columns:
    if col == 'amount':
        print('yes')

In [26]:
y = donors['target_b']
X = donors.drop(['target_b'], axis = 1)

In [28]:
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler().fit(X)
scaled = pd.DataFrame(transformer.transform(X),columns=X.columns)

In [29]:
print(scaled.shape)
print(y.shape)
scaled.head()

(3629, 354)
(3629,)


,tcode,age,income,wealth1,hit,malemili,malevet,vietvets,wwiivets,localgov,stategov,fedgov,wealth2,pop901,pop902,pop903,pop90c1,pop90c2,pop90c3,pop90c4,pop90c5,eth1,eth2,eth3,eth4,eth5,eth6,eth7,eth8,eth9,eth10,eth11,eth12,eth13,eth14,eth15,eth16,age901,age902,age903,age904,age905,age906,age907,chil1,chil2,chil3,agec1,agec2,agec3,agec4,agec5,agec6,agec7,chilc1,chilc2,chilc3,chilc4,chilc5,hhage1,hhage2,hhage3,hhn1,hhn2,hhn3,hhn4,hhn5,hhn6,marr1,marr2,marr3,marr4,hhp1,hhp2,dw1,dw2,dw3,dw4,dw5,dw6,dw7,dw8,dw9,hv1,hv2,hv3,hv4,hu1,hu2,hu3,hu4,hu5,hhd1,hhd2,hhd3,hhd4,hhd5,hhd6,hhd7,hhd8,hhd9,hhd10,hhd11,hhd12,ethc1,ethc2,ethc3,ethc4,ethc5,ethc6,hvp1,hvp2,hvp3,hvp4,hvp5,hvp6,hur1,hur2,rhp1,rhp2,rhp3,rhp4,hupa1,hupa2,hupa3,hupa4,hupa5,hupa6,hupa7,rp1,rp2,rp3,rp4,msa,adi,dma,ic1,ic2,ic3,ic4,ic5,ic6,ic7,ic8,ic9,ic10,ic11,ic12,ic13,ic14,ic15,ic16,ic17,ic18,ic19,ic20,ic21,ic22,ic23,hhas1,hhas2,hhas3,hhas4,mc1,mc2,mc3,tpe1,tpe2,tpe3,tpe4,tpe5,tpe6,tpe7,tpe8,tpe9,pec1,pec2,tpe10,tpe11,tpe12,tpe13,lfc1,lfc2,lfc3,lfc4,lfc5,lfc6,lfc7,lfc8,lfc9,lfc10,occ1,occ2,occ3,occ4,occ5,occ6,occ7,occ8,occ9,occ10,occ11,occ12,occ13,eic1,eic2,eic3,eic4,eic5,eic6,eic7,eic8,eic9,eic10,eic11,eic12,eic13,eic14,eic15,eic16,oedc1,oedc2,oedc3,oedc4,oedc5,oedc6,oedc7,ec1,ec2,ec3,ec4,ec5,ec6,ec7,ec8,sec1,sec2,sec3,sec4,sec5,afc1,afc2,afc3,afc4,afc5,afc6,vc1,vc2,vc3,vc4,anc1,anc2,anc3,anc4,anc5,anc6,anc7,anc8,anc9,anc10,anc11,anc12,anc13,anc14,anc15,pobc1,pobc2,lsc1,lsc2,lsc3,lsc4,voc1,voc2,voc3,hc1,hc2,hc3,hc4,hc5,hc6,hc7,hc8,hc9,hc10,hc11,hc12,hc13,hc14,hc15,hc16,hc17,hc18,hc19,hc20,hc21,mhuc1,mhuc2,ac1,ac2,cardprom,numprom,cardpm12,numprm12,ramntall,ngiftall,cardgift,minramnt,maxramnt,lastgift,timelag,avggift,controln,hphone_d,rfa_2f,cluster2,cluster,datasrce,domain_b,odatew_yr,odatew_mm,dob_yr,dob_mm,minrdate_yr,minrdate_mm,maxrdate_yr,maxrdate_mm,lastdate_yr,lastdate_mm,firstdate_yr,firstdate_mm,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0.000026,0.542553,0.000000,0.111111,0.083333,0.0000,0.316456,0.313131,0.292929,0.083333,0.10,0.016667,0.111111,0.013641,0.014771,0.009874,0.000000,0.0,1.000000,0.494949,0.708333,0.676768,0.323232,0.000000,0.000000,0.010204,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.017544,0.368421,0.513158,0.565789,0.407895,0.552632,0.605263,0.450704,0.373737,0.579710,0.222222,0.175824,0.222222,0.212121,0.53125,0.333333,0.134615,0.081967,0.151515,0.515152,0.447761,0.40,0.181818,0.204082,0.085714,0.195876,0.148936,0.317647,0.595960,0.383838,0.161616,0.070707,0.575758,0.101010,0.162791,0.282609,0.372308,0.442857,0.555556,0.545455,0.000000,0.010101,0.010101,0.010101,0.000000,0.000000,0.000000,0.064694,0.068797,0.153846,0.076923,0.848485,0.161616,0.888889,0.142857,0.111111,0.474747,0.838384,0.616162,0.333333,0.919192,0.090909,0.241379,0.3,0.196429,0.158537,0.171429,0.053571,0.197183,0.434343,0.092784,0.302326,0.253731,0.046512,0.000000,0.000000,0.010101,0.050505,0.292929,0.000000,0.022222,0.383838,0.583333,0.590909,0.278689,0.357143,0.010989,0.000000,0.434343,0.101010,0.000000,0.000000,0.130435,0.000000,0.000000,0.000000,0.151515,0.284261,0.556589,0.646992,0.134000,0.136667,0.167048,0.162672,0.051046,0.412371,0.377358,0.531915,0.186441,0.040404,0.000000,0.000000,0.00,0.000000,0.466667,0.276923,0.262626,0.101010,0.040404,0.000000,0.00,0.000000,0.000000,0.292929,0.367347,0.123711,0.194444,0.343434,0.725275,0.021053,0.737374,0.214286,0.068182,0.000000,0.000000,0.230769,0.000000,0.055556,0.000000,0.000000,0.141304,0.188889,0.444444,0.16,0.565657,0.565657,0.626263,0.515152,0.595960,0.494949,0.606061,0.434343,0.313131,0.585859,0.252525,0.074074,0.019608,0.033333,0.208955,0.088235,0.000000,0.057143,0.192308,0.028571,0.333333,0.763158,0.153846,0.44,0.014286,0.000000,0.196429,0.650794,0.100000,0.000000,0.027778,0.352941,0.075472,0.063830,0.111111,0.000000,0.109091,0.041667,0.055556,0.081081,0.083333,0.02,0.016667,0.112676,0.858586,0.021277,0.000000,0.705882,0.191919,0.464286,0.588235,0.200000,0.096774,0.035714,0.00000

In [31]:
X_train, X_test, y_train, y_test = train_test_split(scaled, y, test_size=0.2, random_state=0)

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

model = LinearRegression()
mean_score = cross_val_score(model, X_train, y_train, cv=10)
print(mean_score)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:

from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)
prediction = lm.predict(X_test)
prediction

# Need to get a target variable for predicting the amount